In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

In [15]:
villeFile="data/villes.csv"
dist_ville = pd.read_csv(villeFile, sep=";", encoding="ISO-8859-1")
population=dist_ville.iloc[:,0].values
n = population.size
k=5
a=0.1
dist= np.nan_to_num(dist_ville.iloc[:, 2:].values)
distance = dist+dist.T
gamma = (1+a)/k * population.sum()

In [16]:
m=Model("mogProjet")
X = m.addMVar((n, k), vtype=GRB.BINARY, name="X")       
m.update()


obj =LinExpr()
obj=0

for i in range(n):
    for j in range(k):
        obj+=distance[i][j]*X[i][j]*population[i]

m.setObjective(obj,GRB.MINIMIZE)

for i in range(n):
    m.addConstr(sum(X[i][j] for j in range(k)) == 1, name="Consville%d" % i)

for j in range(k):
    m.addConstr(sum(X[i][j] * population[i] for i in range(n)) <= gamma, name="ConstEquilibrage%d" % (i))

m.optimize()
   

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 20 rows, 75 columns and 150 nonzeros
Model fingerprint: 0x61ffc24e
Variable types: 0 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+07, 5e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+05]
Found heuristic solution: objective 2.305287e+09
Presolve time: 0.00s
Presolved: 20 rows, 75 columns, 150 nonzeros
Variable types: 0 continuous, 75 integer (75 binary)

Root relaxation: objective 5.357918e+08, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5.357918e+08 5.3579e+08  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processor

In [17]:
print('Valeur de la fonction objectif :', m.objVal/population.sum())     

Valeur de la fonction objectif : 150.31332332725486


In [18]:
X.X.astype(np.uint)

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0]], dtype=uint32)

In [24]:
villeFile="data/villes.csv"
dist_ville = pd.read_csv(villeFile, sep=";", encoding="ISO-8859-1")
population=dist_ville.iloc[:,0].values
n = population.size
k=5
a=0.1
dist= np.nan_to_num(dist_ville.iloc[:, 2:].values)
distance = dist+dist.T
gamma = (1+a)/k * population.sum()

In [25]:
m = Model("mogProjet")
X = m.addMVar((n, n), vtype=GRB.BINARY, name="X")
Y = m.addMVar((n,1),vtype=GRB.BINARY, name="Y")
m.update()


obj =LinExpr()
obj=0

for i in range(n):
    for j in range(n):
        obj+=distance[i][j]*X[i][j]*population[i]

m.setObjective(obj,GRB.MINIMIZE)

for i in range(n):
    m.addConstr(sum(X[i][j] for j in range(n)) == 1, name="Consville%d" % i)

for j in range(n):
        m.addConstr(sum(X[i][j] * population[i] for i in range(n)) <= gamma, name="ConstEquilibrage%d" % (i))

m.addConstr(sum(Y)==k,name="nbSecteur")

for i in range(n):
    for j in range(n):
        m.addConstr(X[i][j]<=Y[j])

m.optimize()
   

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 256 rows, 240 columns and 915 nonzeros
Model fingerprint: 0x8afb05f3
Variable types: 0 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+07, 5e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]
Presolve removed 15 rows and 0 columns
Presolve time: 0.00s
Presolved: 241 rows, 240 columns, 915 nonzeros
Variable types: 0 continuous, 240 integer (240 binary)
Found heuristic solution: objective 1.009971e+09

Root relaxation: objective 4.712254e+08, 80 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.7123e+08    0   26 1.0100e+09 4.7123e+08  53.3%     -    0s
H    0     0                    7.299226e+08 4.7123e+08

In [26]:
Y.x.astype(np.uint)

array([[1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0]], dtype=uint32)

In [27]:
print('Valeur de la fonction objectif :', m.objVal/population.sum())     

Valeur de la fonction objectif : 156.8556237901529


In [28]:
(X.X * distance).astype(int).max()

511